<a href="https://colab.research.google.com/github/pachterlab/kb_docs/blob/main/docs/source/translated/notebooks/virus_detection_sc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Viral proteins in single-cell RNA seq data

[Open in Google Colab](https://colab.research.google.com/github/pachterlab/kb_docs/blob/main/docs/source/translated/notebooks/virus_detection_sc.ipynb)

In this tutorial, we will align single-cell RNA sequencing data collected from Dengue virus (DENV)-infected CEM.NK$^{R}$ cells as well as PBMC data collected from a patient suffering from DENV (during and after infection) to viral RdRP protein sequences. Let's see if we can detect DENV RdRP-like sequences as expected.
  
**Data references:**  
RNA seq data:  
https://www.nature.com/articles/s41598-020-65939-5  
https://www.ncbi.nlm.nih.gov/Traces/study/?acc=PRJNA613041&o=acc_s%3Aa  
PalmDB viral protein reference database:  
https://www.nature.com/articles/s41586-021-04332-2  
  
Written by: Laura Luebbert (last updated: 11/06/2024)

In [1]:
# Number of threads to use during alignments
threads = 2

## Install software

In [2]:
!pip install -q kb_python ffq gget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.5/36.5 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 1

## Download raw sequencing data

In [ ]:
import json
import glob

# Get the FTP download links for raw data using ffq and store the results in a json file
# Here, we will download the data from DENV1 infected CEM.NK^R cells (SRR11321027) and the uninfected control (SRR11321026)
# as well as PBMC data collected from a patient experiencing a DENV infection at fever day -1 (SRR11321527) and -2 (SRR11321526) and 180 (baseline post-infection) (SRR11321528)
!ffq SRR11321027 SRR11321026 SRR11321527 SRR11321526 SRR11321528 \
    --ftp \
    -o ffq.json

# Load ffq output
f = open("ffq.json")
data_json = json.load(f)
f.close()

# Download raw data using FTP links fetched by ffq
for dataset in data_json:
    url = dataset["url"]
    !curl -O $url

[2024-11-06 20:59:30,901]    INFO Parsing run SRR11321027
[2024-11-06 20:59:32,568]    INFO Parsing run SRR11321026
[2024-11-06 20:59:33,994]    INFO Parsing run SRR11321527
[2024-11-06 20:59:35,379]    INFO Parsing run SRR11321526
[2024-11-06 20:59:36,787]    INFO Parsing run SRR11321528
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 42 28.1G   42 11.9G    0     0  35.5M      0  0:13:28  0:05:45  0:07:43 45.3M

## Download the protein reference
In this case, we are using an optimized version of the [PalmDB database](https://github.com/ababaian/palmdb). The database was optimized for the detection of viral proteins from RNA sequencing data as described in [this manuscript](https://www.biorxiv.org/content/10.1101/2023.12.11.571168) and the files are stored in the [accompanying repository](https://github.com/pachterlab/LSCHWCP_2023/tree/main).

In [ ]:
# Download the ID to taxonomy mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/ID_to_taxonomy_mapping.csv
# Download the customized transcripts to gene mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_clustered_t2g.txt
# Download the RdRP amino acid sequences
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_rdrp_seqs.fa

## Build a reference index from the viral protein sequences and mask host (here, human) sequences

In addition to building a reference index from the protein sequences, we also want to mask host (in this case, human) sequences.

Here, we are downloading a precomputed PalmDB reference index for use with `kb` in which the human genome and transcriptome were masked. You can find all available precomputed reference indeces [here](https://github.com/pachterlab/LSCHWCP_2023/tree/main/precomputed_refs).

In [ ]:
# Download precomputed index with masked human genome and transcriptome
!wget https://data.caltech.edu/records/sh33z-hrx98/files/palmdb_human_dlist_cdna_dna.idx?download=1
!mv palmdb_human_dlist_cdna_dna.idx?download=1 palmdb_human_dlist_cdna_dna.idx

**Alternatively, you can compute the PalmDB reference index yourself (and define a different host species):**

The [`--aa` argument](https://kallisto.readthedocs.io/en/latest/translated/pseudoalignment.html) tells `kb` that this is an amino acid reference.

The [`--d-list` argument](https://kallisto.readthedocs.io/en/latest/index/index_generation.html#the-d-list) is the path to the host genome/transcriptome. These sequences will be masked in the index. Here, we are using gget to fetch the latest human genome and transcriptome from Ensembl.

We are using `--workflow custom` here since we do not have a .gtf file for the PalmDB fasta file.

Building the index will take some time (~20 min), since the human genome is quite large.

In [ ]:
# # Build a PalmDB reference index and mask the human genome and transcriptome from scratch

# # Download the human reference genome and transcriptome from Ensembl
# # Replace 'homo_sapiens' here and the genome/transcriptome file names below if the host is a different species
# !gget ref -w cdna,dna -d homo_sapiens

# # Concatenate human genome and transcriptome into one file
# !cat Homo_sapiens.GRCh38.cdna.all.fa.gz Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz > Homo_sapiens.GRCh38.cdna_dna.fa.gz

# %%time
# !kb ref \
#   --workflow custom \
#   --aa \
#   --d-list Homo_sapiens.GRCh38.cdna_dna.fa.gz \
#   -t $threads \
#   -i palmdb_human_dlist_cdna_dna.idx \
#   palmdb_rdrp_seqs.fa

## Align data using kallisto translated search

Create a batch file so we can run all fastq files simultaneously (to learn more about batch files, see Box 7 in the [Protocols paper](https://www.biorxiv.org/content/10.1101/2023.11.21.568164v2.full.pdf)) (to keep track of which cell barcodes belonged to which SRR file, we will activate the `--batch-barcodes` option in the alignment step below):

In [ ]:
import numpy as np

# To decrease the runtime of this example tutorial,
# we will only align the top [n_seq_to_keep] sequences in each fastq file
n_seq_to_keep = 10000000

with open("batch.txt", "w") as batch_file:
  for filename in np.sort(glob.glob("*.fastq.gz")):
    # Shorten fastq (only keep top [n_seq_to_keep] reads)
    n_rows_to_keep = n_seq_to_keep * 4
    filename_short = filename.split(".fastq.gz")[0] + "_short.fastq"
    !zcat $filename | head -$n_rows_to_keep > $filename_short

    # Write batch.txt file
    sample_name = filename.split("/")[-1].split(".")[0]
    batch_file.write(sample_name + "\t" +  filename_short + "\n")

Align fastqs using kallisto translated search:  
The [`-x` argument](https://kallisto.readthedocs.io/en/latest/sc/technologies.html) tells `kb` where to find the barcodes and UMIs in the data. Here, we set it to `10xv2` because this data was generated "using the Chromium Single-Cell 5′ Reagent version 2 kit."

In [ ]:
%%time

!kb count \
    --aa \
    -x 10xv2 \
    -t $threads \
    -i palmdb_human_dlist_cdna_dna.idx \
    -g palmdb_clustered_t2g.txt \
    --h5ad \
    -o kb_output \
    --batch-barcodes \
    batch.txt

# Load the generated count matrix

In [ ]:
!pip install -q kb_python

import anndata
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.colors
%config InlineBackend.figure_format='retina'

def nd(arr):
    """
    Function to transform numpy matrix to nd array.
    """
    return np.asarray(arr).reshape(-1)

In [ ]:
# Open count matrix (AnnData object in h5ad format)
# adata = anndata.read_h5ad("kb_output/counts_unfiltered/adata.h5ad")
adata = anndata.read_h5ad("counts_unfiltered/adata.h5ad")
adata

In [ ]:
# 'barcode' here includes batch/sample barcode (first 16 characters) + cell barcode
adata.obs

Add column showing which SRR file each cell came from using the sample barcodes:

In [ ]:
# Load the sample/batch barcodes
# b_file = open("kb_output/matrix.sample.barcodes")
b_file = open("matrix.sample.barcodes")
barcodes = b_file.read().splitlines()
b_file.close()

# Load the sample/batch names
# s_file = open("kb_output/matrix.cells")
s_file = open("matrix.cells")
samples = s_file.read().splitlines()
s_file.close()

# Create dataframe that maps sample barcodes to sample names
bc2sample_df = pd.DataFrame()
bc2sample_df["sample_barcode"] = barcodes
bc2sample_df["srr"] = samples

# Add sample name to AnnData object
adata.obs["barcode"] = adata.obs.index.values
adata.obs["sample_barcode"] = [bc[:-16] for bc in adata.obs.index.values]
adata.obs["cell_barcode"] = [bc[-16:] for bc in adata.obs.index.values]
adata.obs = adata.obs.merge(bc2sample_df, on="sample_barcode", how="left").set_index("cell_barcode", drop=False)
adata.obs

# Plot counts for Dengue virus (DENV) in each sample

In [ ]:
# Load the PalmDB virus ID to virus taxonomy mapping
u_tax_csv = "ID_to_taxonomy_mapping.csv"
tax_df = pd.read_csv(u_tax_csv)

# Find the reference IDs for Dengue virus RdRPs
# Note: Only the rep_ID (representative ID) will occur in the count matrix
tax_df[tax_df["species"].str.contains("Dengue virus")]

Create bar plot showing raw counts for DENV:

In [ ]:
fig, ax = plt.subplots(figsize=(6, 7))
fontsize = 16
width = 0.75

samples = [
    'SRR11321527',
    'SRR11321526',
    'SRR11321528',
    'SRR11321027',
    'SRR11321026'
    ]
# Label corresponding to each SRR library
x_labels = [
    'Patient PBMC\n(fever day -1)',
    'Patient PBMC\n(fever day -2)',
    'Patient PBMC\n(post-infection control)',
    'DENV-infected CEM.NK$^R$',
    'Uninfected\nControl (CEM.NK$^R$)'
    ]

# Target virus IDs to plot (here, Dengue virus)
target_virus = "Dengue virus"
target_ids = tax_df[tax_df["species"].str.contains(target_virus)]["rep_ID"].unique()

counts = []
labels = samples
for sample in samples:
    counts.append(adata.X[adata.obs["srr"].str.contains(sample), adata.var.index.isin(target_ids)].sum())

x = np.arange(len(labels))

ax.bar(x, counts, width=width, color="#003049", edgecolor="black")

ax.set_yscale("symlog")
ax.set_ylabel("kallisto (raw counts for Dengue virus)", fontsize=fontsize)
# ax.set_xlabel("Sample", fontsize=fontsize)

ax.set_xticks(x, x_labels, rotation=45, ha="right")

ax.tick_params(axis="both", labelsize=fontsize)

ax.grid(True, which="both", color="lightgray", ls="--", lw=1)
ax.set_axisbelow(True)

# Save figure
plt.savefig("denv_rdrp_count.png", dpi=300, bbox_inches="tight")

fig.show()